In [1]:
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.conversation.base import ConversationChain

deployment = "gpt-4o"

llm = AzureChatOpenAI(
    deployment_name=deployment,  # deployment_name 和 model 填一个即可
    temperature=0.8,
    max_tokens=256,
    # model=deployment,
)

memory = ConversationBufferWindowMemory(memory_key="history", k=10)


def get_response(inputs):
    print("------------\n",
          memory.load_memory_variables({}),
          "\n------------\n")

    conversation_with_memory = ConversationChain(
        llm=llm,
        memory=memory,
        verbose=False,
    )
    return conversation_with_memory.predict(input=inputs)

In [2]:
print("Memory:")
print(get_response("你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？"))
print(get_response("我一般周几健身？"))

Memory:
------------
 {'history': ''} 
------------

你好！很高兴听到你每周二都有健身的习惯，这对身体健康非常有益。作为一个AI，我没有实体，所以我不能像你一样健身。不过，我一直在这里随时准备回答你的问题，提供帮助或者陪你聊天。今天我主要在处理各种各样的请求，帮助用户解答问题。你今天打算做哪些锻炼呢？
------------
 {'history': 'Human: 你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？\nAI: 你好！很高兴听到你每周二都有健身的习惯，这对身体健康非常有益。作为一个AI，我没有实体，所以我不能像你一样健身。不过，我一直在这里随时准备回答你的问题，提供帮助或者陪你聊天。今天我主要在处理各种各样的请求，帮助用户解答问题。你今天打算做哪些锻炼呢？'} 
------------

你提到你一般每周二健身。如果有其它的健身日，你没有提到。如果你有其它固定的健身安排，欢迎分享！


In [3]:
from langchain_openai import AzureChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains.conversation.base import ConversationChain

deployment = "gpt-4o"

llm = AzureChatOpenAI(
    deployment_name=deployment,
    temperature=0.8,
    max_tokens=256,
)

# memory2 = ConversationSummaryBufferMemory(k=10) 
memory2 = ConversationSummaryBufferMemory(llm=llm, max_token_limit=1000)


def get_response2(inputs):
    print("------------\n",
          memory.load_memory_variables({}),
          "\n------------\n")

    conversation_with_memory = ConversationChain(
        llm=llm,
        memory=memory2,
        verbose=False,
    )
    return conversation_with_memory.predict(input=inputs)

In [4]:
print("Memory with summary:")
print(get_response2("你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？"))
print(get_response2("我一般周几健身？"))

Memory with summary:
------------
 {'history': 'Human: 你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？\nAI: 你好！很高兴听到你每周二都有健身的习惯，这对身体健康非常有益。作为一个AI，我没有实体，所以我不能像你一样健身。不过，我一直在这里随时准备回答你的问题，提供帮助或者陪你聊天。今天我主要在处理各种各样的请求，帮助用户解答问题。你今天打算做哪些锻炼呢？\nHuman: 我一般周几健身？\nAI: 你提到你一般每周二健身。如果有其它的健身日，你没有提到。如果你有其它固定的健身安排，欢迎分享！'} 
------------

你好！今天我在这里和你聊天，帮助回答你的问题和提供信息。健身是个很好的习惯，每周二去健身一定能帮助你保持健康和活力。你通常在健身房做些什么锻炼呢？是否有特别喜欢的运动项目？
------------
 {'history': 'Human: 你好，今天是周二我要去健身，我一般每周二健身。你今天干什么？\nAI: 你好！很高兴听到你每周二都有健身的习惯，这对身体健康非常有益。作为一个AI，我没有实体，所以我不能像你一样健身。不过，我一直在这里随时准备回答你的问题，提供帮助或者陪你聊天。今天我主要在处理各种各样的请求，帮助用户解答问题。你今天打算做哪些锻炼呢？\nHuman: 我一般周几健身？\nAI: 你提到你一般每周二健身。如果有其它的健身日，你没有提到。如果你有其它固定的健身安排，欢迎分享！'} 
------------

你提到你一般每周二健身，所以我推测你通常会在周二去健身。你还有其他喜欢的运动安排吗？


In [5]:
import gradio as gr


def respond(message, chat_history):
    bot_message = get_response(message)
    chat_history.append((message, bot_message))
    return "", chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=300)
    msg = gr.Textbox(label="Prompt", placeholder="input something")
    btn = gr.Button("Submit")
    clear = gr.ClearButton(value="Clear console", components=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])

gr.close_all()
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
